In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.metrics import mean_squared_error
import numpy as np
import lightgbm as lgb

# 1. 資料加載
X_train = pd.read_csv("X_train.csv")
y_train = pd.read_csv("y_train.csv")
X_test = pd.read_csv("X_test.csv")

# 2. 識別類別型特徵
categorical_columns = X_train.select_dtypes(include=["object"]).columns
# print(f"類別型特徵: {categorical_columns}")

# 3. 對所有類別型特徵進行編碼
for col in categorical_columns:
    encoder = LabelEncoder()
    X_train[col] = encoder.fit_transform(X_train[col])
    X_test[col] = X_test[col].map(
        lambda x: encoder.transform([x])[0] if x in encoder.classes_ else -1
    )

# 創建衍生特徵
X_train["每平方公尺價格"] = X_train["建物移轉總面積平方公尺"] / X_train["土地移轉總面積平方公尺"]
X_train["樓層比例"] = X_train["移轉層次"] / X_train["總樓層數"]
X_test["每平方公尺價格"] = X_test["建物移轉總面積平方公尺"] / X_test["土地移轉總面積平方公尺"]
X_test["樓層比例"] = X_test["移轉層次"] / X_test["總樓層數"]

# 添加交互特徵
X_train["建物型態_主要用途"] = X_train["建物型態"].astype(str) + "_" + X_train["主要用途"].astype(str)
X_test["建物型態_主要用途"] = X_test["建物型態"].astype(str) + "_" + X_test["主要用途"].astype(str)

# 對交互特徵進行編碼
encoder = LabelEncoder()
X_train["建物型態_主要用途"] = encoder.fit_transform(X_train["建物型態_主要用途"])
X_test["建物型態_主要用途"] = X_test["建物型態_主要用途"].map(
    lambda x: encoder.transform([x])[0] if x in encoder.classes_ else -1
)

# 特徵標準化
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)


# 訓練驗證分割
X_train_split, X_val_split, y_train_split, y_val_split = train_test_split(
    X_train_scaled, y_train["單價元平方公尺"], test_size=0.2, random_state=42
)

# 模型選擇與訓練（使用最佳超參數）
lgb_model = lgb.LGBMRegressor(
    boosting_type='gbdt',
    n_estimators=500,
    learning_rate=0.05,
    max_depth=7,
    num_leaves=31,
    random_state=87,
    verbose=-1
)
lgb_model.fit(X_train_split, y_train_split)

# 驗證集 RMSE
y_val_pred = lgb_model.predict(X_val_split)
rmse = np.sqrt(mean_squared_error(y_val_split, y_val_pred))
print(f"LightGBM Validation RMSE: {rmse}")

# 測試集預測
y_test_pred = lgb_model.predict(X_test_scaled)

# 儲存結果
submission = pd.DataFrame({"Id": X_test["Id"], "單價元平方公尺": y_test_pred})
submission.to_csv("r13725052_周哲群.csv", index=False)

LightGBM Validation RMSE: 32764.929948820118


In [19]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.metrics import mean_squared_error
import lightgbm as lgb
import numpy as np

# 1. 加載資料
# 讀取訓練集 (X_train)、目標值 (y_train)、測試集 (X_test)
X_train = pd.read_csv('X_train.csv')
y_train = pd.read_csv('y_train.csv')
X_test = pd.read_csv('X_test.csv')

# 2. 類別型特徵的編碼
# 對特徵進行編碼，例如「鄉鎮市區」、「交易標的」等文字型資料轉換為數值
categorical_columns = ["鄉鎮市區", "交易標的", "都市土地使用分區", "建物型態", "主要用途", "主要建材", "路名"]
for col in categorical_columns:
    if col in X_train.columns:
        encoder = LabelEncoder()
        # 將訓練集和測試集中該列的類別進行數值編碼
        X_train[col] = encoder.fit_transform(X_train[col].astype(str))
        X_test[col] = X_test[col].map(
            lambda x: encoder.transform([x])[0] if x in encoder.classes_ else -1
        )

# 3. 日期特徵的處理
# 提取建築完成年份，計算距今年份
X_train["建築完成年月"] = X_train["建築完成年月"].astype(str)
X_test["建築完成年月"] = X_test["建築完成年月"].astype(str)
X_train["建築完成年份"] = 2024 - pd.to_numeric(X_train["建築完成年月"].str[:4], errors='coerce')
X_test["建築完成年份"] = 2024 - pd.to_numeric(X_test["建築完成年月"].str[:4], errors='coerce')
# 填補缺失值，使用中位數替代
X_train["建築完成年份"].fillna(X_train["建築完成年份"].median(), inplace=True)
X_test["建築完成年份"].fillna(X_test["建築完成年份"].median(), inplace=True)

# 4. 新增衍生特徵
# 計算土地面積與建物面積的比例作為新特徵
X_train["土地建物面積比例"] = X_train["土地移轉總面積平方公尺"] / (X_train["建物移轉總面積平方公尺"] + 1e-5)
X_test["土地建物面積比例"] = X_test["土地移轉總面積平方公尺"] / (X_test["建物移轉總面積平方公尺"] + 1e-5)

# 5. 增加環境特徵的密度
# 計算環境特徵（如地鐵站、超商、公園）的密度，這些特徵除以土地面積
for col in ["地鐵站", "超商", "公園"]:
    X_train[f"{col}_密度"] = X_train[col] / X_train["土地移轉總面積平方公尺"]
    X_test[f"{col}_密度"] = X_test[col] / X_test["土地移轉總面積平方公尺"]

# 6. 高斯核距離特徵
# 計算每筆資料到市中心的距離，作為新特徵
target_coord = {"lat": 25.0330, "long": 121.5654}  # 市中心座標（假設值）
X_train["到市中心距離"] = np.sqrt((X_train["縱坐標"] - target_coord["lat"])**2 +
                               (X_train["橫坐標"] - target_coord["long"])**2)
X_test["到市中心距離"] = np.sqrt((X_test["縱坐標"] - target_coord["lat"])**2 +
                              (X_test["橫坐標"] - target_coord["long"])**2)

# 7. 保留數值型特徵
# 確保模型只使用數值型特徵進行訓練
numerical_columns = X_train.select_dtypes(include=[np.number]).columns
X_train = X_train[numerical_columns]
X_test = X_test[numerical_columns]

# 8. 特徵標準化
# 使用 StandardScaler 對數值特徵進行標準化，讓不同範圍的特徵值保持一致
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# 9. 訓練驗證集劃分
# 將訓練集拆分為訓練子集和驗證集，用於模型評估
X_train_split, X_val_split, y_train_split, y_val_split = train_test_split(
    X_train_scaled, y_train["單價元平方公尺"], test_size=0.2, random_state=42
)

# 10. 模型訓練
# 使用 LightGBM 訓練模型，設置適當的超參數
lgb_model = lgb.LGBMRegressor(
    boosting_type='gbdt',
    n_estimators=500,
    learning_rate=0.05,
    max_depth=7,
    num_leaves=31,
    random_state=87,
    verbose=-1
)
lgb_model.fit(X_train_split, y_train_split)

# 11. 模型評估
# 在驗證集上預測並計算 RMSE 作為評估指標
y_val_pred = lgb_model.predict(X_val_split)
rmse = np.sqrt(mean_squared_error(y_val_split, y_val_pred))
print(f"Validation RMSE: {rmse}")

# 12. 測試集預測
# 使用訓練好的模型對測試集進行預測
y_test_pred = lgb_model.predict(X_test_scaled)

# 13. 保存結果
# 將測試集預測結果保存為提交格式的 CSV 文件
submission = pd.DataFrame({"Id": X_test["Id"], "單價元平方公尺": y_test_pred})
submission.to_csv('r13725052_周哲群.csv', index=False)

print("模型訓練與預測完成，結果已保存至 'r13725052_周哲群.csv'")


C:\Users\user\AppData\Local\Temp\ipykernel_7376\3058494912.py:33: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  X_train["建築完成年份"].fillna(X_train["建築完成年份"].median(), inplace=True)
C:\Users\user\AppData\Local\Temp\ipykernel_7376\3058494912.py:34: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as 

Validation RMSE: 32781.575045822494
模型訓練與預測完成，結果已保存至 'r13725052_周哲群.csv'
